In [ ]:
import sqlite3
import pandas as pd
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt 
import seaborn as sns # Import seaborn

from datetime import datetime
from datetime import date
from dateutil import parser

from collections import defaultdict

import warnings
import time

In [ ]:
pd.options.display.max_columns = None

In [ ]:
# open connection 
conn = sqlite3.connect("data/database.sqlite")
cur = conn.cursor()

In [ ]:
# function to execute queries
def executeQuery(cur, query):
    print("executing query: ")
    cur.execute(query)
    return cur.fetchall()

In [ ]:
# needs to be broken into smaller functions and call those individually on different tables
def eda_for_df(data):
    print(data.shape)
    print(data.columns)
    print(data.dtypes)
    
    missing_values = data.isnull().sum()
    print(missing_values)
    
    summary_statistics = data.describe()
    print(summary_statistics)

    # Histograms
    for column in data.columns:
        if data[column].dtype in ['int64', 'float64']:
            plt.figure()
            sns.histplot(data[column])
            plt.title(column)
            plt.show()
            
    # Box plots
    for column in data.columns:
        if data[column].dtype in ['int64', 'float64']:
            plt.figure()
            sns.boxplot(data[column])
            plt.title(column)
            plt.show()
            
    correlation_matrix = data.corr()
    plt.figure(figsize=(16, 12))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.show()
    
#     sns.pairplot(data, vars=['column1', 'column2', 'column3'])
#     plt.show()
    
    # Bar plots
    for column in data.columns:
        if data[column].dtype == 'object':
            plt.figure()
            data[column].value_counts().plot.bar()
            plt.title(column)
            plt.show()

    # Count plots
    for column in data.columns:
        if data[column].dtype == 'object':
            plt.figure()
            sns.countplot(data=data, x=column)
            plt.title(column)
            plt.show()

In [ ]:
# list of all tables
q_all_tables = """SELECT name FROM sqlite_master
    WHERE type='table';"""
all_tables = executeQuery(cur, q_all_tables)
print(all_tables)

In [ ]:
# read the Match table into Pandas DataFrame
q_matches = "SELECT * FROM MATCH;"
df_matches = pd.read_sql_query(q_matches, conn)
# eda_for_df(df_matches)
# df_matches.info()

In [ ]:
q_player = "SELECT * FROM PLAYER;"
df_player = pd.read_sql_query(q_player, conn)
df_player.head()

In [ ]:
q_player_attr = "SELECT * FROM Player_Attributes;"
df_player_attr = pd.read_sql_query(q_player_attr, conn)
df_player_attr.head()

In [ ]:
q_team = "SELECT * FROM Team"
df_team = pd.read_sql_query(q_team, conn)
df_team.head()

In [ ]:
q_country = "SELECT * FROM Country"
df_country = pd.read_sql_query(q_country, conn)
df_country.head()

In [ ]:
q_league = "SELECT * FROM League"
df_league = pd.read_sql_query(q_league, conn)
df_league.head()

In [ ]:
q_team_attr = "SELECT * FROM Team_Attributes"
df_team_attr = pd.read_sql_query(q_team_attr, conn)
df_team_attr.head()